<a href="https://colab.research.google.com/github/Saurx9611/Fintech/blob/main/Fintech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import kagglehub
path = kagglehub.dataset_download("wordsforthewise/lending-club")

Using Colab cache for faster access to the 'lending-club' dataset.


In [6]:
print("Path to dataset files:", path)
import os
csv_file = os.path.join(path, 'accepted_2007_to_2018Q4.csv.gz')

Path to dataset files: /kaggle/input/lending-club


In [7]:
rej_cols = ['Amount Requested', 'Risk_Score', 'Debt-To-Income Ratio', 'State', 'Employment Length']
rej = pd.read_csv(os.path.join(path, 'rejected_2007_to_2018Q4.csv.gz'), usecols=rej_cols, low_memory=False)

In [8]:
rej.head()

,Amount Requested,Risk_Score,Debt-To-Income Ratio,State,Employment Length
0,1000.0,693.0,10%,NM,4 years
1,1000.0,703.0,10%,MA,< 1 year
2,11000.0,715.0,10%,MD,1 year
3,6000.0,698.0,38.64%,MA,< 1 year
4,1500.0,509.0,9.43%,MD,< 1 year


In [9]:
acc_cols = ['loan_amnt', 'last_fico_range_low', 'dti', 'addr_state', 'emp_length']
acc = pd.read_csv(
    os.path.join(path, 'accepted_2007_to_2018Q4.csv.gz'),
    usecols=acc_cols,
    low_memory=False
)

In [10]:
acc.head()

,loan_amnt,emp_length,addr_state,dti,last_fico_range_low
0,3600.0,10+ years,PA,5.91,560.0
1,24700.0,10+ years,SD,16.06,695.0
2,20000.0,10+ years,IL,10.78,700.0
3,35000.0,10+ years,NJ,17.06,675.0
4,10400.0,3 years,PA,25.37,700.0


In [11]:
rej.columns = ['amount', 'fico', 'dti', 'state', 'emp_length']
rej['Target'] = 0

In [12]:
rej.head()

,amount,fico,dti,state,emp_length,Target
0,1000.0,693.0,10%,NM,4 years,0
1,1000.0,703.0,10%,MA,< 1 year,0
2,11000.0,715.0,10%,MD,1 year,0
3,6000.0,698.0,38.64%,MA,< 1 year,0
4,1500.0,509.0,9.43%,MD,< 1 year,0


In [13]:
rej['dti'] = pd.to_numeric(rej['dti'].astype(str).str.replace('%', ''), errors='coerce')
rej['emp_length'] = rej['emp_length'].astype(str).str.extract(r'(\d+)').astype(float).fillna(0)
rej.head()

,amount,fico,dti,state,emp_length,Target
0,1000.0,693.0,10.00,NM,4.0,0
1,1000.0,703.0,10.00,MA,1.0,0
2,11000.0,715.0,10.00,MD,1.0,0
3,6000.0,698.0,38.64,MA,1.0,0
4,1500.0,509.0,9.43,MD,1.0,0


In [14]:
acc.columns = ['amount', 'emp_length', 'state', 'dti', 'fico']
acc['Target'] = 1
acc['emp_length'] = acc['emp_length'].astype(str).str.extract(r'(\d+)').astype(float).fillna(0)
acc.head()

,amount,emp_length,state,dti,fico,Target
0,3600.0,10.0,PA,5.91,560.0,1
1,24700.0,10.0,SD,16.06,695.0,1
2,20000.0,10.0,IL,10.78,700.0,1
3,35000.0,10.0,NJ,17.06,675.0,1
4,10400.0,3.0,PA,25.37,700.0,1


In [15]:
df_final = pd.concat([
    rej.sample(50000, random_state=42),
    acc.sample(50000, random_state=42)
], axis=0)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
df_final.head()

,amount,fico,dti,state,emp_length,Target
0,10000.0,695.0,22.41,NY,1.0,1
1,3500.0,725.0,10.58,TX,1.0,1
2,14000.0,746.0,24.90,NY,0.0,0
3,8000.0,785.0,19.66,CA,3.0,1
4,10000.0,735.0,8.35,FL,1.0,1


In [17]:
le=LabelEncoder()
df_final['state']=le.fit_transform(df_final['state'])
df_final.head()

,amount,fico,dti,state,emp_length,Target
0,10000.0,695.0,22.41,34,1.0,1
1,3500.0,725.0,10.58,43,1.0,1
2,14000.0,746.0,24.90,34,0.0,0
3,8000.0,785.0,19.66,4,3.0,1
4,10000.0,735.0,8.35,9,1.0,1


In [26]:
df_final = df_final.fillna(0)
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
X=df_final.drop('Target',axis=1)
y=df_final['Target']

In [28]:
md=RandomForestClassifier(n_estimators=100,max_depth=10,random_state=42)
md.fit(X,y)


RandomForestClassifier(max_depth=10, random_state=42)

In [32]:
import joblib
joblib.dump(md, 'loan_approval_model.pkl')
joblib.dump(le, 'state_encoder.pkl')

['state_encoder.pkl']